In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

print(sys.path)

['/home/tiendat/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/torchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [2]:
import torch
import torch.nn as nn
import math
import argparse
from dataPreprocess import DatasetPreprocess
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import os
import torch.optim as optim
import time
import warnings
import torch.nn.functional as F
import time
from torchsummary import summary

In [3]:
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

#================================================================
root_dir = "../road/predict_fab_multi/TFRecord_w15_s15/1/"
window_size = 15
d_model = 12
max_time_position = 10000
gran = 1e-7
log_e = 2
batch_size = 10

train_dataset = DatasetPreprocess(root_dir, window_size, window_size, d_model, max_time_position, gran, log_e, is_train=True)
test_dataset = DatasetPreprocess(root_dir, window_size,window_size, d_model, max_time_position, gran, log_e, is_train=False)


print("TRAIN SIZE:", len(train_dataset), " TEST SIZE:", len(test_dataset), " SIZE:", len(train_dataset)+len(test_dataset), " TRAIN RATIO:", round(len(train_dataset)/(len(train_dataset)+len(test_dataset))*100), "%")

# 2 DataLoader? 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)
print('finish load data')

TRAIN SIZE: 84688  TEST SIZE: 21188  SIZE: 105876  TRAIN RATIO: 80 %
finish load data


In [4]:
class Config:
    def __init__(self, args):
        self.model_name = 'IDS-Transformer_' + args.type
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.payload_size = 8 
            
        self.mode = args.mode
        self.dout_mess = 10
        self.d_model = self.dout_mess
        self.nhead = 10 
        
        self.tse = args.tse
        self.pad_size = args.window_size # 15
        self.window_size =  args.window_size # 15
        self.max_time_position = 10000
        self.num_layers = 6
        self.gran = 1e-6
        self.log_e = 2
        
        if args.type == 'chd':
            self.classes_num = 5
        elif args.type == 'road_mas':
            self.classes_num = 6
        else: # road_fab
            self.classes_num = 7 
        
            
        self.batch_size = args.batch_size
        self.epoch_num = args.epoch
        self.lr = args.lr
        self.root_dir = args.indir
        
        # self.root_dir = './data/Processed/TFRecord_w29_s29/2/'
        # self.root_dir = './road/predict_fab_multi/TFRecord_w15_s15/1/'
        # self.root_dir = './road/predict_mas/TFRecord_w15_s15/4/'
        self.model_save_path = '../model/' + self.model_name + '/'
        if not os.path.exists(self.model_save_path):
            os.mkdir(self.model_save_path)
        self.result_file = '/home/tiendat/transformer-entropy-ids/result/trans8_performance.txt'

        self.isload_model = False  
        self.start_epoch = 24  # The epoch of the loaded model
        self.model_path = 'model/' + self.model_name + '/' + self.model_name + '_model_' + str(self.start_epoch) + '.pth' 

class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor
    
class Autoencoder1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Autoencoder1D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, output_dim)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return F.relu(x)

class Time_Positional_Encoding(nn.Module):
    def __init__(self, embed, max_time_position, device):
        super(Time_Positional_Encoding, self).__init__()
        self.device = device

    def forward(self, x, time_position):
        out = x.permute(1, 0, 2)
        out = out + nn.Parameter(time_position, requires_grad=False).to(self.device)
        out = out.permute(1, 0, 2)
        return out
    
class TransformerPredictor(nn.Module):
    def __init__(self, config):
        super(TransformerPredictor, self).__init__()
        
        self.ae = Autoencoder1D(12, config.dout_mess).to(config.device)
        self.pad_size = config.pad_size
        
        self.position_embedding = Time_Positional_Encoding(config.d_model, config.max_time_position, config.device).to(config.device)
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=config.d_model, nhead=config.nhead).to(config.device)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=config.num_layers).to(config.device)
        self.fc = nn.Linear(config.d_model, config.classes_num).to(config.device)
        
    def forward(self, header, sl_sum, mask, time_position):
        x = torch.concat((header, sl_sum), dim=-1)
        
        ae_out = torch.empty((x.shape[0], 10, 0)).to(config.device)
        for i in range(self.pad_size):
            tmp = self.ae(x[:, i, :]).unsqueeze(2)
            ae_out = torch.concat((ae_out, tmp), dim=2)
        x = ae_out.permute(2, 0, 1)
            
        out = self.position_embedding(x, time_position)
        out = self.transformer_encoder(out, src_key_padding_mask=mask)
        out = out.permute(1, 0, 2)
        out = torch.sum(out, 1)
        out = self.fc(out)
        return out

args = argparse.Namespace(
    indir="",
    window_size=15,
    type="road_mas",
    epoch=200,
    batch_size=32,
    lr=0.0001,
    mode="cb",
    tse=True
)

config = Config(args)
model = TransformerPredictor(config)
start_epoch = -1
loss_func = nn.CrossEntropyLoss().to(config.device)
opt = optim.Adam(model.parameters(), lr=config.lr)
lr_scheduler = CosineWarmupScheduler(opt, warmup=50, max_iters=config.epoch_num*len(train_loader))

/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import time
import torch

# Assuming 'inputs' is your input data
inputs = [torch.randn(10, 15, 4), torch.randn(10, 15, 8), torch.randn(10, 15), torch.randn(10, 15, 10)]

# Move the model and inputs to the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
inputs = [i.to(device) for i in inputs]

# Make sure the model is in evaluation mode
model.eval()

# Warm up (for more accurate timing)
with torch.no_grad():
    _ = model(*inputs)

# Time the inference
start_time = time.time()
with torch.no_grad():
    _ = model(*inputs)
end_time = time.time()

print(f'Inference time: {end_time - start_time} seconds')

Inference time: 0.005384206771850586 seconds


In [6]:
from thop import profile

x = torch.rand((10, 15, 4)).to(config.device)
y = torch.rand((10, 15, 8)).to(config.device)
z = torch.rand((10, 15,)).to(config.device)
a = torch.rand((10, 15, 10)).to(config.device)

macs, params = profile(model, inputs=[x, y, z, a])
print('MACs (G): ', macs/1000**2)
print('Params (M): ', params/1000**2)

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
MACs (G):  37.095
Params (M):  0.259528
